In [43]:
# 二级模型单条测试
import os
import copy
import numpy as np
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import tensorflow.keras.backend as k
import tensorflow.keras as keras
from hanlp.transform.tsv import TSVTaggingTransform
from model_main import build_model, load_vocabs, build

save_dir = "save_dir/"
model_dir = "model_dir/0121_best_model/"

def load_model(model_dir):
    transform = TSVTaggingTransform()
    # 读取字典
    load_vocabs(transform, save_dir)
    # # 构建模型
    # model=build(transform)
    # # 加载参数
    # model.load_weights(model_dir + "model.h5")
    # return model, transform
    return transform

In [34]:
transform = load_model(model_dir)
model = tf.keras.models.load_model(model_dir,compile=False)

In [24]:
def predict(pre_str_list, model_):
    pre_chars = []
    for s in pre_str_list:
        pre_chars.append(list(s))
#     pre_chars = [list(pre_str)]
    data, flat = transform.input_to_inputs(pre_chars)
    dataset = transform.inputs_to_dataset(data, batch_size=512)
    for idx, batch in enumerate(dataset):
        X = batch[0]
        Y = model_.predict_on_batch(batch)
        return Y
def process(address):
    ad = address.split(',')
    newAd = []
    for i in range(len(ad)):
        if i%2==0:
            newAd.append(ad[i])
    return "".join(newAd)
def predict_bilstm(pre_str_list, model_, dense_model, first=True):
    pre_chars = []
    for s in pre_str_list:
        ad = process(s)
        pre_chars.append(list(ad))
#     pre_chars = [list(pre_str)]
    data, flat = transform.input_to_inputs(pre_chars)
    dataset = transform.inputs_to_dataset(data, batch_size=512)
    for idx, batch in enumerate(dataset):
        print(batch)
        first_second = []
        first = []
        second = []
        X = batch[0]
        Y = model_.predict_on_batch(batch)
        Y_first = copy.deepcopy(Y)
        Y_second = copy.deepcopy(Y)
#         Y[:, :, 301:] = Y[:, :, 301:] *2
#         Y[:, :, :300] = Y[:, :, :300] *2
        Y_first[:, :, 301:] = 0
        Y_second[:, :, :300] = 0
#         Y_second[:, :, :] = Y_second[:, :, :] *100000
        
        Y_dense = dense_model.predict_on_batch(Y)
        Y_dense_first = dense_model.predict(Y_first)
        Y_dense_second = dense_model.predict(Y_second)
        
        for output in transform.Y_to_outputs(Y_dense, X=X, inputs=data):
            first_second.append(output)
        for output in transform.Y_to_outputs(Y_dense_first, X=X, inputs=data):
            first.append(output)
        for output in transform.Y_to_outputs(Y_dense_second, X=X, inputs=data):
            second.append(output)
        return first_second, first, second

In [35]:
def build_model_dense(transform) -> tf.keras.Model:
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(transform.tag_vocab)), name='dense'))
    return model

def get_bimodel_dense_model(model, transform):
    bimodel = tf.keras.models.Model(model.input, model.get_layer('bilstm').output)
    dense_model = build_model_dense(transform)
    dense_model.build((None, None, 600))
    dense_model.set_weights(model.get_layer('dense').get_weights())
    return bimodel, dense_model

In [36]:
bimodel, dense_model = get_bimodel_dense_model(model, transform)

In [37]:
a = predict(["卢屋荔校园村荔园工业路"], bimodel)

In [59]:
# sentence = "广东省东莞市常平镇广东省东莞市常平镇卢屋荔校园村荔园工业路8号3栋"
# first_second, first, second = predict_bilstm([sentence], bimodel, dense_model, False)
# print(first_second[0])
# print(first[0])
# print(second[0])
sen = np.array([[13,   73,  178,   73,  219,   20,  674,  126,  201,   13,   73,
         178,   73,  219,   20,  674,  126,  201, 2128, 1910, 1288,  620,
          57,  128, 1288,   57,   55,   56,    4,  133,   15,  187,  858, 0, 0]])
for output in transform.Y_to_outputs(model.predict_on_batch((sen)), X=sen, inputs=sen):
    print(output)


['X', 'X', 'R1', 'X', 'X', 'R2', 'X', 'X', 'R4', 'X', 'X', 'R1', 'X', 'X', 'R2', 'X', 'X', 'R4', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'R5', 'X', 'R6', 'X', 'R6', '<pad>', '<pad>']


In [23]:
from model import WindowEmbedding
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
def build_model_dense(transform) -> tf.keras.Model:
    S_inputs = Input(shape=(None,), dtype='int32')
    embeddings = WindowEmbedding(4, input_dim=len(transform.word_vocab), output_dim=100,
                                            trainable=True, mask_zero=True)(S_inputs)
    model = Model(inputs=S_inputs, outputs=embeddings)
    return model

In [11]:
def get_bimodel_dense_model(model, transform):
    bimodel = tf.keras.models.Model(model.input, model.get_layer('bilstm').output)
    dense_model = build_model_dense(transform)
    dense_model.build((None,))
    dense_model.set_weights(model.get_layer('dense').get_weights())
    return bimodel, dense_model

In [25]:
ebed_model = build_model_dense(transform)
ebed_model.build((None,))
ebed_model.set_weights(model.get_layer('window_embedding_1').get_weights())

In [100]:
def predict_bilstm(pre_str_list, ebed_model, first=True):
#     pre_chars = []
#     for s in pre_str_list:
#         ad = process(s)
#         pre_chars.append(list(ad))
    pre_chars = pre_str_list
#     pre_chars = [list(pre_str)]
    data, flat = transform.input_to_inputs(pre_chars)
    dataset = transform.inputs_to_dataset(data, batch_size=512)
    a = []
    for idx, batch in enumerate(dataset):
        first_second = []
        first = []
        second = []
        X = batch[0]
        Y = ebed_model.predict_on_batch(batch)
#         Y_second[:, :, :] = Y_second[:, :, :] *100000
        a.append(Y)
    return a

In [64]:
def predict(pre_str_list, model):
    pre_chars = []
    for s in pre_str_list:
        pre_chars.append(list(s))
#     pre_chars = [list(pre_str)]
    data, flat = transform.input_to_inputs(pre_chars)
    dataset = transform.inputs_to_dataset(data, batch_size=512)
    for idx, batch in enumerate(dataset):
        X = batch[0]
        Y = model.predict_on_batch(batch)
        return [i for i in Y]
        for output in transform.Y_to_outputs(Y, X=X, inputs=data):
            yield output

In [69]:
pre_str_list = ['河北省,R1,石家庄市,R2,元氏县,R3,河北省,R1,石家庄市,R2,元氏县槐南工业区东张乡政府,R24,西200米,R90',
             '广东省,R1,佛山市,R2,南海区,R3,广东省,R1,佛山市,R2,南海区,R3,丹灶镇,R4,良登竹径工业区,R31,南横一路,R5,6号,R6',
            '广东省,R1,佛山市,R2,南海区,R3,广东省,R1,佛山市,R2,南海区,R3,九江镇,R4,石江工业区,R31,三马路,R5',
            '广东省,R1,佛山市,R2,南海区,R3,广东省,R1,佛山市,R2,南海区,R3,里水镇,R4,大冲展旗工业区,R31,1路6号,R6',
            '广东省,R1,佛山市,R2,顺德区,R3,佛山市顺德区,R3,勒流街道,R4,江义工业区,R31,东六路,R5,2号,R6',
            '广东省,R1,佛山市,R2,顺德区,R3,佛山市,R2,顺德区,R3,容桂容边,R5,天河工业区,R31,七路,R5,12号,R6',
            '广东省,R1,佛山市,R2,顺德区,R3,广东省,R1,佛山市,R2,顺德区,R3,北滘,R4,黄涌工业区,R31,西一南8号,R6']
temp = predict_bilstm(pre_str_list, ebed_model)[0]

In [101]:
pre_str_list = ['广','东','省','佛','山','市','顺','德','区','佛','山','市','顺','德','区','勒',
                '流','街','道','江','义','工','业','区','东','六','路','2','号']
temp_1 = predict_bilstm(pre_str_list, ebed_model)[0]

In [102]:
temp_1

array([[[ 0.00891354, -0.03209693, -0.12876877, ..., -0.10749534,
          0.07192606, -0.03572354],
        [ 0.10977469,  0.02123746,  0.00441451, ..., -0.01792312,
         -0.0536033 , -0.01791018],
        [-0.01430568, -0.09100763, -0.05758253, ..., -0.17678417,
          0.12066743,  0.0044687 ],
        ...,
        [-0.04916796,  0.10606522,  0.0121194 , ...,  0.        ,
          0.        ,  0.        ],
        [-0.06024643, -0.03333642, -0.05882998, ...,  0.        ,
          0.        ,  0.        ],
        [-0.1320741 , -0.02835562, -0.00468848, ...,  0.        ,
          0.        ,  0.        ]]], dtype=float32)

In [108]:
temp_1[0][:2]

array([[ 8.91353749e-03, -3.20969336e-02, -1.28768772e-01,
         2.06360663e-03,  9.80240479e-02, -1.90828830e-01,
        -2.05077417e-02,  2.89460309e-02, -2.67456342e-02,
        -1.93425536e-03, -2.07433687e-03,  8.17551166e-02,
        -1.78046167e-01, -1.20620783e-02, -2.88388003e-02,
        -1.49016529e-01, -2.75254726e-01,  2.12058555e-02,
        -2.02332795e-01, -7.03666732e-02, -3.83252613e-02,
        -1.78544804e-01, -2.77575273e-02, -7.13360263e-03,
         1.24860369e-01, -3.21176797e-02,  2.90381275e-02,
         7.48023316e-02, -4.23178039e-02, -8.60829651e-02,
         5.17199039e-02,  3.50697935e-02,  1.58995688e-02,
        -1.14172690e-01,  2.92038899e-02,  1.03048630e-01,
        -5.97549565e-02,  3.02523654e-02, -3.86497602e-02,
         6.43073954e-03,  4.41473201e-02,  6.57338202e-02,
         4.78459448e-02, -5.22832312e-02,  1.34404441e-02,
         1.97335377e-01, -1.05891794e-01, -6.92015886e-02,
        -1.75058655e-02, -1.08622476e-01, -1.06083795e-0

In [85]:
pre_str_list = [['广'],['东'],['省'],['佛'],['山'],['市'],['顺'],['德'],['区'],['佛'],['山'],['市'],['顺'],['德'],['区'],['勒'],['流'],
                ['街'],['道'],['江'],['义'],['工'],['业'],['区'],['东'],['六'],['路'],['2']['号']]
temp_2 = predict_bilstm(pre_str_list, ebed_model)[0]

In [86]:
temp_2

array([[[-0.06073404, -0.07354487, -0.06179835,  0.05081944,
         -0.0656313 , -0.05243554,  0.00578966, -0.07309459,
          0.0115547 ,  0.0456668 ,  0.03127453,  0.00568552,
          0.10800829,  0.06137731, -0.02345406, -0.02982301,
         -0.11747639, -0.04171754, -0.05582061,  0.01802774,
         -0.06074217, -0.05268715, -0.05020398,  0.03121414,
         -0.05853981, -0.00696356,  0.06872517,  0.08160995,
          0.02126663,  0.07887636, -0.09575915, -0.21584806,
          0.02179018,  0.07383625,  0.066004  ,  0.06343392,
          0.00848005, -0.00336216,  0.01821843,  0.0550901 ,
          0.01076857, -0.04901535, -0.05013582, -0.03012724,
         -0.06644308,  0.08146847, -0.02272104, -0.0550751 ,
         -0.02378508,  0.08203981,  0.05101835,  0.09937834,
         -0.03068127,  0.01180447, -0.09671542,  0.0268044 ,
          0.05982529, -0.00692189,  0.02512457, -0.01176721,
          0.05019813, -0.01023135, -0.01328981, -0.00971175,
         -0.05140851, -0